In [1]:
## Learning while working LSTM models

# Cross Entropy Loss function works well with Label Encoder whereaas Binary Cross Entorpy is used for multi class classification.


# RNN LSTM Model for BinarMulti Class  Classification using Attention

In [1]:
 #Import All Libraries Here
import pandas as pd
from sklearn.metrics import accuracy_score ,  roc_curve, auc , classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np

from sklearn.preprocessing import OneHotEncoder

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split , RandomizedSearchCV


from imblearn.over_sampling import SMOTE 
from imblearn.over_sampling import RandomOverSampler


from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
import optuna

import time

# PyTorch Import

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

from torch.utils.data import WeightedRandomSampler

c:\Users\shashi.vish\Python Environment\RNA_ModX\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Record Constants
INPUT_TRAIN_IN = "../../../data/train_in.csv"
INPUT_TRAIN_OUT = "../../../data/train_out.csv"  
INPUT_TEST_IN = "../../../data/test_in.csv"
INPUT_TEST_OUT = "../../../data/test_out.csv"
INPUT_VALIDATION_IN = "../../../data/valid_in_nucleo.csv"
INPUT_VALIDATION_OUT  = "../../../data/valid_out.csv"

TARGET_MODEL_PATH = '../../webapp/model_files'


WINDOW_SIZE =  50 # Final RNA Sequence will be 101 Length 
MAX_LENGTH = (WINDOW_SIZE*2) +1

# 1 - One Hot Encoding with Pytorch in build Emnedding 
# 2 - K-mers with Word2Vec
ENCODING_METHOD = 1


# 1 - LSTM with Cross Entropy 
MODEL = 1


FRAMEWORK = "PYTORCH"


PERFORM_DATA_BALANCING = True
K_MERS_SIZE = 3
EMBEDDING_DIMENSION  = 3

# Startegy to Crop Sequene
# MID - Modification is present at Mid of cropped Sequence 
# END - Modification is present at End of cropepd Sequence 
CROP_STRATEGY = 'END'

# Y Category Encoding Method
# LABEL or ONE_HOT 
TARGET_ENCODING = 'LABEL'


In [3]:
#Read X Varaibles and Y Varaibles

x_train_raw =  pd.read_csv(INPUT_TRAIN_IN, header=None , skiprows=1)
y_train_raw =  pd.read_csv(INPUT_TRAIN_OUT, header=None , skiprows=1)

x_test_raw =  pd.read_csv(INPUT_TEST_IN, header=None , skiprows=1 )
y_test_raw =  pd.read_csv(INPUT_TEST_OUT, header=None , skiprows=1)

x_valid_raw =  pd.read_csv(INPUT_VALIDATION_IN, header=None , skiprows=1 )
y_valid_raw =  pd.read_csv(INPUT_VALIDATION_OUT, header=None , skiprows=1 )


middle_index = (x_train_raw.shape[1] // 2) + 1

if CROP_STRATEGY == 'MID':
    STRAT_INEDX =middle_index - WINDOW_SIZE -1 
    END_INDEX =middle_index + WINDOW_SIZE 

if CROP_STRATEGY == 'END':
    STRAT_INEDX =middle_index - (WINDOW_SIZE*2) -1 
    END_INDEX =middle_index


In [4]:
print(f"Train Shape of X : {x_train_raw.shape} and Tranin Shape of Y : {x_train_raw.shape}")
print(f"Test Shape of X : {x_test_raw.shape} and Test Shape of Y : {y_test_raw.shape}")
print(f"Validation Shape of X : {x_valid_raw.shape} and Validation Shape of Y : {y_valid_raw.shape}")

Train Shape of X : (304661, 1001) and Tranin Shape of Y : (304661, 1001)
Test Shape of X : (1200, 1001) and Test Shape of Y : (1200, 12)
Validation Shape of X : (3599, 1001) and Validation Shape of Y : (3599, 12)


# Encode Y Category

In [5]:
## Filter Dataset to Keep only Target Binary Class 

RMs = ['hAm','hCm','hGm','hTm','hm1A','hm5C','hm5U','hm6A','hm6Am','hm7G','hPsi','Atol','NonMoD']
RMEncoding = [12,1,2,3,4,5,6,7,8,9,10,11,0]

def convert_y_to_original_labels(row):
    label = ""
    for index , n in enumerate(row.tolist()) :
        if n == 1 :
            label = RMs[index]
    if label == '':
        return 'NonMoD'
    return label

def get_original_y_lables( y_data ):
    # Convert One Hot Encoded Y to to Original Labels 
    y_original_labels = y_data.apply(convert_y_to_original_labels,axis=1)
    return y_original_labels  



### Encode X  - Vanilla One Hot Encoding

In [6]:
number_of_unique_kmers = set()
def encode_seq(kmer_token):

    # A 1 0 0 0
    # C 0 1 0 0
    # T/U 0 0 0 1
    # G 0 0 1 0
    # N 0 0 0 0

    encoding_dict = {
        'A': [1, 0, 0, 0],
        'C': [0, 1, 0, 0],
        'G': [0, 0, 1, 0],
        'T': [0, 0, 0, 1],
        'U': [0, 0, 0, 1],
        'N': [0, 0, 0, 0],
    }

    encoded_sequence = []
    number_of_unique_kmers.add(kmer_token)
    for  base in kmer_token:
        encoded_sequence.append(encoding_dict[base])
    return np.array(encoded_sequence).flatten()

def applyOneHotEncoding(tokenized_sequences):
    encoded_sequences = []
    for seq in tokenized_sequences:
        encoded_sequences.append(encode_seq(seq)) 
 
    return np.array(encoded_sequences).flatten()


def encode_with_one_hot_encoding(x_train_raw):
    truncated_df =  x_train_raw.iloc[:,STRAT_INEDX :END_INDEX] # Window Starts from V501 with 50 window size
    concatenated_column= truncated_df.apply(lambda row: ''.join(map(str, row)), axis=1)
    df_result = truncated_df.assign(Sequence=concatenated_column)
    tokenized_sequences =  df_result['Sequence'].apply(applyOneHotEncoding).tolist()
    return tokenized_sequences


### K-mers with Word2Vec

In [7]:
# Model Documentation - 
# Encode RNA sequences using Word2Vec embeddings
import gensim

all_unique_k_mers = set()

def encode_sequence(sequence, word2vec_model):
    embeddings = []
    for kmer in sequence:
        embeddings.append(word2vec_model.wv[kmer])
    return np.array(embeddings)

def applyKmersAndEncoding(seq):
    k = 3
    tokenized_sequences = [''.join(seq[i:i+k]) for i in range(0, len(seq)-k+1)]
    return tokenized_sequences

def encode_x_with_word2Vec(x_train_raw):
    truncated_df =  x_train_raw.iloc[:,STRAT_INEDX :END_INDEX] # Window Starts from V501 with 50 window size
    truncated_df['Sequence'] = truncated_df.apply(lambda row: ''.join(map(str, row)), axis=1)
    tokenized_sequences =  truncated_df['Sequence'].apply(applyKmersAndEncoding).tolist()

    flat_list = [item for sublist in tokenized_sequences for item in sublist]
    all_unique_k_mers.update(flat_list)

    # The result, tokenized_sequences, is a list of lists, where each inner list
    #  contains the k-mers of the corresponding RNA sequence from the truncated_df list. 

    # Train Word2Vec model on tokenized_sequences (list of lists) instead of unique k-mers
    word2vec_model = gensim.models.Word2Vec(tokenized_sequences, vector_size=EMBEDDING_DIMENSION, window=5, min_count=1, workers=4)
    
    result = []
    for seq in tokenized_sequences:
        embedding = encode_sequence(seq, word2vec_model)
        result.append(embedding)
    return np.array(result)


### Encode Y

In [8]:
def encode_target(y_data):
    if TARGET_ENCODING  == 'LABEL':
        le = LabelEncoder()
        y_encoded = le.fit_transform(y_data)
        #y_encoded = torch.tensor(y_encoded, dtype=torch.float32)

    if TARGET_ENCODING == 'ONE_HOT':
        ohe = OneHotEncoder(sparse=False)
        y_encoded = ohe.fit_transform(y_data.to_numpy().reshape(-1, 1))
        y_encoded = torch.tensor(y_encoded, dtype=torch.float32)

    return y_encoded

In [10]:
#test = np.array(encode_x_with_word2Vec(x_train_raw))
# print("Input Dimension : " , len(all_unique_k_mers))
# print("Shape of X  " , test[0][0])

## Prepare Data 

In [9]:
y_train_original =  get_original_y_lables(y_train_raw)
y_test_original =  get_original_y_lables(y_test_raw )
y_valid_original =  get_original_y_lables(y_valid_raw )

print(y_train_original.value_counts())

if ENCODING_METHOD ==1:
    X_train = torch.tensor(encode_with_one_hot_encoding(x_train_raw) , dtype=torch.long)
    X_test = torch.tensor(encode_with_one_hot_encoding(x_test_raw) , dtype=torch.long)
    X_valid = torch.tensor(encode_with_one_hot_encoding(x_valid_raw) , dtype=torch.long)

if ENCODING_METHOD == 2:
    X_train = torch.tensor(encode_x_with_word2Vec(x_train_raw) , dtype=torch.float32)
    X_test = torch.tensor(encode_x_with_word2Vec(x_test_raw) , dtype=torch.float32)
    X_valid = torch.tensor(encode_x_with_word2Vec(x_valid_raw) , dtype=torch.float32)


print("Encoding Completed for X Features.")
y_train = encode_target(y_train_original)
y_test = encode_target(y_test_original)
y_valid = encode_target(y_valid_original)


#Caste to Torch Tensor
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)
y_valid = torch.tensor(y_valid, dtype=torch.long)

print("Encoding Completed For Y Features.")

# Calculate class weights
class_weights = 1.0 / torch.bincount(y_train)
print(class_weights)
weights = class_weights[y_train]
sampler = WeightedRandomSampler(weights, len(weights))


NonMoD    152208
hm6A       64978
Atol       52418
hm1A       16141
hm5U        3496
hm5C        3007
hPsi        2937
hm6Am       2247
hTm         2053
hCm         1678
hAm         1391
hGm         1271
hm7G         836
Name: count, dtype: int64


C:\Users\shashi.vish\AppData\Local\Temp\ipykernel_30164\23543330.py:8: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:248.)
  X_train = torch.tensor(encode_with_one_hot_encoding(x_train_raw) , dtype=torch.long)


Encoding Completed for X Features.
Encoding Completed For Y Features.
tensor([1.9077e-05, 6.5700e-06, 7.1891e-04, 5.9595e-04, 7.8678e-04, 3.4048e-04,
        4.8709e-04, 6.1954e-05, 3.3256e-04, 2.8604e-04, 1.5390e-05, 4.4504e-04,
        1.1962e-03])


In [28]:
X_train = X_train.to(torch.float32)
X_test = X_test.to(torch.float32)
X_valid = X_valid.to(torch.float32)
X_train.dtype

torch.float32

### Hyper Parameters

In [10]:
## LSTM Hyper Paramerts 

hyperparameter = {}

if ENCODING_METHOD == 1:
    hyperparameter['INPUT_DIMENSION'] =  len(X_train[0]) #4 # For One Hot Encoding Input Dimension would be 4 as there only 4 unique nucleocide 
if ENCODING_METHOD ==2 :
     hyperparameter['INPUT_DIMENSION'] = EMBEDDING_DIMENSION # Dimension of Word2Vec Model

hyperparameter['HIDDEN_DIMENSION'] = 3
hyperparameter['NO_OF_LAYERS'] = 3
hyperparameter['BATCH_SIZE'] = 32
hyperparameter['OUTPUT_DIMENSION'] = 13
hyperparameter['EMBEDDING_DIMENSION'] = EMBEDDING_DIMENSION # if you are using Word2Vec Encoding then this should be same as Word2Vec Embedding Dim 
hyperparameter['DROP_OUT'] = 0.5

### LSTM Model with Binary Classification (Works with Label Encoder)

In [13]:
# class RNAClassifierWithWord2Vec(nn.Module):
#     def __init__(self, input_dim,embedding_dim, hidden_dim,num_layers, output_dim , dropout=0.5):
#         super(RNAClassifierWithWord2Vec, self).__init__()
        
#         #Bi-Directional LSTM Model
#         self.lstm = nn.LSTM(embedding_dim, hidden_dim,num_layers, batch_first=True , bidirectional=True)

#         # Attention Layer - 2 * hidden dimension since we are using bi-directional lstm
#         self.attention = BahdanauAttention(2 * hidden_dim)

#         #Fully Connected Layer   
#         self.fc = nn.Linear(2*hidden_dim, output_dim) 

#         # Drop out layer for Overfitting 
#         self.dropout = nn.Dropout(dropout)


#     def forward(self, x):

#         lstm_out, (h,c) = self.lstm(x)

#         # Concatenate the hidden states from both directions
#         h = torch.cat((h[-2, :, :], h[-1, :, :]), dim=1)

#         # Calculate Context Vector and Attention Weight
#         context_vector , attention_weights =  self.attention(h , lstm_out)
        
#         # Pass it to drop out layer 
#         context_vector = self.dropout(context_vector)
        
#         # Finally pass it to fully connected layer.
#         out = self.fc(context_vector)
        
#         return out # Sequeez to Single dimension for loss calculation

In [23]:

class RNADataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        return self.X[index], self.y[index]

class BahdanauAttention(nn.Module):
    def __init__(self, hidden_dim):
        super(BahdanauAttention, self).__init__()
        self.W = nn.Linear(hidden_dim, hidden_dim)
        self.v = nn.Linear(hidden_dim, 1)

    def forward(self, hidden, encoder_outputs):
        hidden = hidden.unsqueeze(1)
        hidden = hidden.expand(-1, encoder_outputs.size(1), -1)
        alignment_scores = self.v(torch.tanh(self.W(hidden + encoder_outputs)))
        attention_weights = F.softmax(alignment_scores, dim=1)
        context_vector = torch.sum(attention_weights * encoder_outputs, dim=1)
        return context_vector, attention_weights

class RNAClassifierWihtoutEmbedding(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim, dropout=0.5):
        super(RNAClassifierWihtoutEmbedding, self).__init__()

        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, bidirectional=True)
        self.attention = BahdanauAttention(2 * hidden_dim)
        self.fc = nn.Linear(2 * hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        lstm_out, (h, c) = self.lstm(x)
        h = torch.cat((h[-2, :, :], h[-1, :, :]), dim=1)
        context_vector, attention_weights = self.attention(h, lstm_out)
        context_vector = self.dropout(context_vector)
        out = self.fc(context_vector)
        return out


class RNAClassifier(nn.Module):
    def __init__(self, input_dim,embedding_dim, hidden_dim,num_layers, output_dim , dropout=0.5):
        super(RNAClassifier, self).__init__()

        #Pytroch Embedding 
        self.embedding = nn.Embedding(input_dim,embedding_dim)

        #Bi-Directional LSTM Model
        self.lstm = nn.LSTM(embedding_dim, hidden_dim,num_layers, batch_first=True , bidirectional=True)
        #self.lstm = nn.LSTM(input_dim, hidden_dim,num_layers, batch_first=True , bidirectional=True)

        # Attention Layer - 2 * hidden dimension since we are using bi-directional lstm
        self.attention = BahdanauAttention(2 * hidden_dim)

        #Fully Connected Layer   
        self.fc = nn.Linear(2*hidden_dim, output_dim) 

        # Drop out layer for Overfitting 
        self.dropout = nn.Dropout(dropout)


    def forward(self, x):

        x  = x.long()

        x_embeded =  self.embedding(x)
        
        lstm_out, (h,c) = self.lstm(x_embeded)

        # Concatenate the hidden states from both directions
        h = torch.cat((h[-2, :, :], h[-1, :, :]), dim=1)

        # Calculate Context Vector and Attention Weight
        context_vector , attention_weights =  self.attention(h , lstm_out)
        
        # Pass it to drop out layer 
        context_vector = self.dropout(context_vector)
        
        # Finally pass it to fully connected layer.
        out = self.fc(context_vector)
        
        return out # Sequeez to Single dimension for loss calculation
    
def validate_model(model, test_dataloader , device ,loss_function):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    true_labels = []
    predicted_labels = []

    with torch.no_grad():
        for inputs, labels in test_dataloader:
            inputs = inputs.to(device)
            labels = labels.long().to(device)

            outputs = model(inputs)
            loss = loss_function(outputs, labels)

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            true_labels.extend(labels.cpu().numpy())  # Capture True Lables for Summary Report
            predicted_labels.extend(predicted.cpu().numpy()) # Capture Predicted Labels Lables for Summary Report

    validation_loss = running_loss / len(test_dataloader)
    validation_accuracy = correct / total

    return validation_loss, validation_accuracy , true_labels , predicted_labels


def train_model(model , train_dataloader , test_dataloader ,device ,epochs, optimizer ,loss_function):
    prev_accuracy = 0.0
    stagnant_epochs = 0

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0

        for i, (inputs, labels) in enumerate(train_dataloader):
            start_time = time.time()
            inputs  = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs) 
            
            loss = loss_function(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            
        epoch_loss = running_loss / len(train_dataloader)
        val_loss, val_acc , true_labels , predicted_labels= validate_model(model, test_dataloader ,device, loss_function)

                # Check if overall accuracy has improved
        if val_acc <= prev_accuracy:
            stagnant_epochs += 1
        else:
            stagnant_epochs = 0

        # If accuracy hasn't improved for 3 epochs, stop training
        if stagnant_epochs >= 3:
            print("Training stopped: Overall accuracy has not increased for last 3 epochs")
            break

        # Update previous accuracy
        prev_accuracy = val_acc

        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f"Epoch {epoch + 1}, Train Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f} , Time: {elapsed_time:.2f} seconds")


In [29]:

print("Started Model Training..")

# Create dataset and dataloader
train_dataset = RNADataset(X_train, y_train)
test_dataset = RNADataset(X_test, y_test)
valid_dataset = RNADataset(X_valid, y_valid)


train_dataloader = DataLoader(train_dataset, batch_size=hyperparameter['BATCH_SIZE'], sampler=sampler)
test_dataloader = DataLoader(test_dataset, batch_size=hyperparameter['BATCH_SIZE'], shuffle=False)
valid_dataloader = DataLoader(valid_dataset, batch_size=hyperparameter['BATCH_SIZE'], shuffle=False)



# Initialize the model, loss function, and optimizer

if ENCODING_METHOD == 1:
    model = RNAClassifier(input_dim=hyperparameter['INPUT_DIMENSION'],
                                embedding_dim=hyperparameter['EMBEDDING_DIMENSION'], 
                                hidden_dim=hyperparameter['HIDDEN_DIMENSION'] , 
                                num_layers = hyperparameter['NO_OF_LAYERS'],
                                output_dim=hyperparameter['OUTPUT_DIMENSION'],
                                dropout=hyperparameter['DROP_OUT'] )

# if ENCODING_METHOD ==2 :
#     model = RNAClassifierWithWord2Vec(input_dim=hyperparameter['INPUT_DIMENSION'],
#                                 embedding_dim=hyperparameter['EMBEDDING_DIMENSION'], 
#                                 hidden_dim=hyperparameter['HIDDEN_DIMENSION'] , 
#                                 num_layers = hyperparameter['NO_OF_LAYERS'],
#                                 output_dim=hyperparameter['OUTPUT_DIMENSION'],
#                                 dropout=hyperparameter['DROP_OUT'] )



loss_function = nn.CrossEntropyLoss()  ## MSELoss of Regression problem  # BCELoss for binary classification
optimizer = optim.Adam(model.parameters() ,  lr=0.01)

# Number of Parameters for Model
total_parameters = []
for p in model.parameters():
    total_parameters.append(p.numel())

print(f"Total Number of Parameters for Model Training : { sum(total_parameters)} " )

# Train the model
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# GPU uncomment below line for parallel computation.
#model = nn.DataParallel(model)

print(model)
print("Model Parameters  : " , hyperparameter)

# Train Model with configured Parameter
train_model(model, train_dataloader ,test_dataloader, device ,num_epochs,optimizer,loss_function)


Started Model Training..
Total Number of Parameters for Model Training : 11696 
RNAClassifier(
  (embedding): Embedding(404, 3)
  (lstm): LSTM(404, 3, num_layers=3, batch_first=True, bidirectional=True)
  (attention): BahdanauAttention(
    (W): Linear(in_features=6, out_features=6, bias=True)
    (v): Linear(in_features=6, out_features=1, bias=True)
  )
  (fc): Linear(in_features=6, out_features=13, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)
Model Parameters  :  {'INPUT_DIMENSION': 404, 'HIDDEN_DIMENSION': 3, 'NO_OF_LAYERS': 3, 'BATCH_SIZE': 32, 'OUTPUT_DIMENSION': 13, 'EMBEDDING_DIMENSION': 3, 'DROP_OUT': 0.5}


IndexError: too many indices for tensor of dimension 2

### Calculate Model Accuracy

In [ ]:
# Evaluate the model on the test dataset
_, final_accuracy, true_labels, predicted_labels = validate_model(model, valid_dataloader,device,loss_function)

# Print the final accuracy
print(f"Final Accuracy: {final_accuracy:.4f}")

# Print the classification summary
print("\n Classification Summary:")
print(classification_report(true_labels, predicted_labels))

### Optuna Hyperparameter Training

In [ ]:
import optuna

def objective(trial):
    # Suggest hyperparameters
    embedding_dim = trial.suggest_int("embedding_dim", 16, 300)
    hidden_dim = trial.suggest_int("hidden_dim", 32, 512)
    num_layers = trial.suggest_int("num_layers", 1, 4)
    num_epochs = 100
    dropout = trial.suggest_float("dropout", 0.1, 0.5)
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-4, 1e-2)
    batch_size = trial.suggest_categorical("batch_size", [256, 512, 1024])

    # Create dataset and dataloader
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

    # Initialize the model, loss function, and optimizer
    model = RNAClassifier(hyperparameter['INPUT_DIMENSION'], embedding_dim, hidden_dim, num_layers, hyperparameter['OUTPUT_DIMENSION'], dropout)
    loss_function = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Train the model
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    # GPU uncomment below line for parallel computation.
    model = nn.DataParallel(model)
    train_model(model, train_dataloader, test_dataloader, device, num_epochs, optimizer, loss_function)

    # Evaluate the model
    _, val_acc, _, _ = validate_model(model, valid_dataloader, device, loss_function)

    return val_acc

# Create an Optuna study
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

# Print the best hyperparameters
print("Best trial:")
trial = study.best_trial
print(" Value: {:.4f}".format(trial.value))
print(" Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))


### LSTM Model with Binary Cross Entropy Loss (Works with One Hot Encoder)

In [ ]:
# ## Define LSTM and Train

# class RNADataset(Dataset):
#     def __init__(self, X, y):
#         self.X = X
#         self.y = y

#     def __len__(self):
#         return len(self.X)

#     def __getitem__(self, index):
#         return self.X[index], self.y[index]

# class RNAClassifier_BCE(nn.Module):
#     def __init__(self, unqique_k_mer_size, embedding_dim,hidden_dim,num_layers, output_dim):
#         super(RNAClassifier_BCE, self).__init__()
#         self.embedding  = nn.Embedding(unqique_k_mer_size,embedding_dim)
#         self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
#         self.fc = nn.Linear(hidden_dim, output_dim)

#     def forward(self, x):

#         x_embeded = self.embedding(x)
#         _, (hidden, _) = self.lstm(x_embeded)

#        # print("Hidden Dimansion : " , hidden.shape)
#         x = self.fc(hidden[-1]) # Squeeze does not make any differecent in Tensor , basically removing 0 demsion from tensor 

#         # When we squeeze a tensor, the dimensions of size 1 are removed. 
#         # The elements of the original tensor are arranged with the remaining dimensions. For example, 
#         # if the input tensor is of shape: (m×1×n×1) then the output tensor after squeeze will be of shape: (m×n). 

#         # print("Ouput  Dimansion : " ,x.shape)      

#         # print("Result After Applygin Softmax : " , torch.softmax(x, dim=-1) )                                      
#         return x



# if MODEL==1 :
#     print("Excuting LSTM Model")

#     # Create dataset and dataloader
#     print("X Shape of Tensor : " , X_train.shape , " Y Tensor Shape : " , y_train.shape)
#     print("HyperParameter for Training : ", hyperparameter)
#     train_dataset = RNADataset(X_train, y_train)
#     test_dataset = RNADataset(X_test, y_test)

#     train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
#     test_dataloader = DataLoader(test_dataset, batch_size=hyperparameter['BATCH_SIZE'], shuffle=False)

#     # Calculating INPUT_DIMENSION for LSTM Mode 
#     # If you are just using One Hot then INPUT_DIMENSION would be 4 because there are 4 possible unique nucleosides
#     # But if you are using K mers then INPUT_DIMENSION would be number of unique k-mers on all dataset

#     # LSTM model Format - (batch_size, sequence_length, input_size)  
#     # batch_size comes from DataLoaded , you can tune it.
#     # Sequence Length - Lenth of X[0] shape (32, 19, 12)  Here 19 is sequence length , 
#     # Input Size = 12 length of X[0][0] - Basically all unique nucleoside or k-mers

#     # Initialize the model, loss function, and optimizer
#     # model = RNAClassifier_BCE(input_size=4788, 
#     #                           hidden_size=hyperparameter['HIDDEN_DIMENSION'] , 
#     #                           num_layers = hyperparameter['NO_OF_LAYERS'],
#     #                           output_size=hyperparameter['OUTPUT_DIMENSION'] )
#     model = RNAClassifier_BCE(unqique_k_mer_size=hyperparameter['INPUT_DIMENSION'],
#                               embedding_dim=hyperparameter['EMBEDDING_DIMENSION'] 
#                               hidden_dim=hyperparameter['HIDDEN_DIMENSION'] , 
#                               num_layers = hyperparameter['NO_OF_LAYERS'],
#                               output_dim=hyperparameter['OUTPUT_DIMENSION'] )
    
#     criterion = nn.CrossEntropyLoss()
#     optimizer = optim.Adam(model.parameters())

#     # Number of Parameters for Model
#     total_parameters = []
#     for p in model.parameters():
#         total_parameters.append(p.numel())
    
#     print(f"Total Number of Parameters for Model Training : { sum(total_parameters)} " )

#     # Train the model
#     num_epochs = 10
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     model = model.to(device)
    
#     for epoch in range(num_epochs):
#         model.train()
#         running_loss = 0.0
#         correct = 0
#         total = 0

#         for i, (inputs, labels) in enumerate(train_dataloader):
#             inputs, labels = inputs.to(device), labels.to(device)

#             print("Target Labesl : " , labels.shape)
#             print("Ouput Labesl : " , outputs.shape)

#             optimizer.zero_grad()
#             outputs = model(inputs)
#             loss = criterion(outputs, labels)
#             loss.backward()
#             optimizer.step()

#             running_loss += loss.item()
            
#         epoch_loss = running_loss / len(train_dataloader)
#         print(f"Epoch {epoch + 1}, Loss: {epoch_loss:.4f}")




In [ ]:
model.eval()
correct_predictions = 0
total_predictions = 0

with torch.no_grad():
    for x_batch, y_batch in test_dataloader:
        outputs = model(x_batch)
        
        # Basically models predict probability for each class and then finally we pick up with hightest probabiliyt class 
        # and then compare with actual class to calculate accuracy. 
        
        ## Predicted torch.max(outputs, 1) 
#         values=tensor([0.7304, 0.4554, 0.3984, 0.4091, 0.9271, 0.4370, 0.3837, 0.4395, 0.4005,
#         0.5451, 0.5642, 0.6866, 0.5768, 0.5160, 0.5093, 0.4947, 0.6823, 0.4612,
#         0.4073, 0.5041, 0.6982, 0.4116, 0.4092, 0.5589, 0.6010, 0.4693, 0.6324,
#         0.5524, 0.4339, 0.7889, 0.5216, 0.6588]),
#         indices=tensor([ 1, 10, 10, 10,  1, 10, 10, 10,  1, 10,  1,  1,  0,  1,  1, 10,  1, 10,
#         10, 10, 10,  1, 10,  0, 10,  1,  1,  0, 10,  1,  1,  0]))

        _, predicted = torch.max(outputs, 1)  # Get the class index with the highest probability
        #print(torch.max(outputs, 1))
        true_labels = torch.max(y_batch, 1).indices  # Get the true class indices from the one-hot encoded labels [0,1,0,0,,,... ] Get index of 1
        correct_predictions += (predicted == true_labels).sum().item()
        total_predictions += y_batch.size(0)

accuracy = correct_predictions / total_predictions


print(f"Validation Accuracy: {accuracy:.4f}")
